In [1]:
import numpy as np
import tensorflow as tf
import time
import random

/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint

In [2]:
# gpu_available = tf.test.is_gpu_available()
# is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
# is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
# is_cuda_gpu_min_3

In [5]:
train = np.load('../data/freebase15k_numpy/train.npy')
validation = np.load('../data/freebase15k_numpy/validation.npy')
test = np.load('../data/freebase15k_numpy/test.npy')

entities = np.load('../data/freebase15k_numpy/entities.npy')
relations = np.load('../data/freebase15k_numpy/relations.npy')

train = tf.convert_to_tensor(train.astype(dtype=np.int16), dtype=tf.int16)
validation = tf.convert_to_tensor(validation.astype(dtype=np.int16), dtype=tf.int16)
test = tf.convert_to_tensor(test.astype(dtype=np.int16), dtype=tf.int16)
relations = tf.convert_to_tensor(relations.astype(dtype=np.int16), dtype=tf.int16)
entities = tf.convert_to_tensor(entities.astype(dtype=np.float32), dtype=tf.float32)

In [58]:
iteration = tf.constant(500)
dim = tf.constant(100)
kns = tf.constant(5)
alpha = tf.Variable(0.000001, dtype = tf.float32)
beta = tf.Variable(0.0000001, dtype = tf.float32)
z = tf.constant(0.5)
nn1 = tf.random.uniform( shape=(entities.shape[0], dim),
      minval=tf.math.truediv(z , tf.cast(dim,tf.float32),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float32)),
      dtype=tf.dtypes.float32, seed=None, name='nn1')
      
nn0 =tf.random.uniform(shape=(entities.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float32)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float32)),
      dtype=tf.dtypes.float32, seed=None, name='nn0')
nn2 = tf.random.uniform(shape=(relations.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float32)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float32)),
      dtype=tf.dtypes.float32, seed=None, name='nn2')

In [ ]:
def calculateRank(triple):   
    selectedEntities = entities[:,0]
    if(len((np.where((train[:,1] == triple[1]) & (train[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , train[np.where((train[:,1] ==triple[1]) & (train[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((test[:,1] == triple[1]) & (test[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , test[np.where((test[:,1] ==triple[1]) & (test[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((validation[:,1] == triple[1]) & (validation[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , validation[np.where((validation[:,1] ==triple[1]) & (validation[:,2] ==triple[2]))][0], invert =True ))[0]
    couraptedH = np.empty((selectedEntities.shape[0],3))
    couraptedH[:,0] =selectedEntities
    couraptedH[:,1] =triple[1]
    couraptedH[:,2] =triple[2]
    
    selectedEntities = entities[:,0]
    if(len((np.where((train[:,0] == triple[0]) & (train[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , train[np.where((train[:,0] == triple[0]) & (train[:,2] == triple[2]))][0], invert =True ))[0]
    if(len((np.where((test[:,0] == triple[0]) & (test[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , test[np.where((test[:,0] ==triple[0]) & (test[:,2] ==triple[2]))][0], invert =True ))[0]
    if(len((np.where((validation[:,0] == triple[0]) & (validation[:,2] == triple[2]))[0]))> 0):
        selectedEntities = np.where(np.isin(selectedEntities , validation[np.where((validation[:,0] ==triple[0]) & (validation[:,2] ==triple[2]))][0], invert =True ))[0]
    couraptedT = np.empty((selectedEntities.shape[0],3))
    couraptedT[:,0] = triple[0]
    couraptedT[:,1] = selectedEntities
    couraptedT[:,2] =triple[2]
    courapted = np.concatenate((couraptedH, couraptedT), axis=0)
    courapted = courapted.astype(np.int16)
    courapted = np.vstack([courapted, triple])
    
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(tf.convert_to_tensor(nn1[courapted[:,1]],dtype=tf.float32),tf.convert_to_tensor(tf.transpose(tf.convert_to_tensor(nn0[courapted[:,0]],dtype=tf.float32) + tf.convert_to_tensor(nn2[triple[2]],dtype=tf.float32) ),dtype=tf.float32),axes=1 )))
    
    courapted =np.concatenate((courapted, p[:, None]), axis=1)
    courapted = courapted[np.argsort(courapted[:, -3])]
    return np.where((courapted[:,0] == triple[0]) & (courapted[:,1] == triple[1]))[0][0]

In [ ]:
def mrr(ranks):
    inverse = []
    for rank in ranks:
        inverse.append(float(1)/rank)
    summ = sum(inverse)
    return float(1)/len(inverse) * summ

In [59]:
x = tf.random.categorical([entities[:,2]], 5, dtype=None, seed=None, name=None)
paddings = tf.constant([[0, 1,], [0, 0]])
my_goal = tf.pad(x, paddings, constant_values=0)
my_goal =  tf.cast(my_goal, tf.int32) 
sa = tf.constant([[122],[1]])

trans = tf.transpose(tf.concat(
    [my_goal, sa], 1, name='concat'
))
tf.gather(nn1, trans[:,0])
dott =tf.tensordot(
   tf.gather(nn1, trans[:,0]) , nn0[5], 1, name=None
)
a = tf.cast(trans[:,1], tf.float32)
alpha *(a- dott)


<tf.Tensor: id=1212, shape=(6,), dtype=float32, numpy=
array([-2.4999995e-09, -2.4999995e-09, -2.4999995e-09, -2.4999995e-09,
       -2.4999995e-09,  9.9750002e-07], dtype=float32)>

In [18]:
# start = time.time()
for x in range(0 , iteration):
    tf.random.shuffle(train, seed=None, name=None)
    for triple in train:
        negative_samples = [(int(target), 0) for target in (tf.random.categorical([entities[:,2]], kns, dtype=None, seed=None, name=None))]
        classifiers = [(triple[1], 1)] + negative_samples
        # random.shuffle(classifiers)
        neu1e = np.zeros(dim)
        for target, label in classifiers:
            _target_index = target
            _head_index = triple[0]
            _relation_index = triple[2]
            z = np.dot(nn1[_target_index], nn0[_head_index] + nn2[_relation_index])
            p = sigmoid(z)
            g = alpha * (label - p)
            g1 = beta * (label - p)
            neu1e += (g * nn1[_target_index] + g1 * nn2[_relation_index])
            nn1[_target_index] += g * nn0[_head_index]
            nn2[_relation_index] += g * nn0[_head_index]
    # print('total one sample calc time =>  ' , time.time() - start)       
    # if x % 10 == 0 and x > 20:
# _mrrList = []
# # for triple in validation:
# _mrrList.append(calculateRank(validation[0,:]))
# print('total one sample calc time =>  ' , time.time() - start) 

TypeError: only size-1 arrays can be converted to Python scalars